---
title:  "Ordered_Logistic_Regression_PyMC3"
excerpt: "Ordered_Logistic_Regression"

categories:
  - Machine-Learning
tags:
  - PyMC3  
  - Medium  
  - 순서형 로지스틱 Regression    
last_modified_at: 2020-09-19T22:00:00-05:00
---

## Reference  
- Elizaveta Semenova Medium Blog (https://medium.com/@liza_p_semenova/ordered-logistic-regression-and-probabilistic-programming-502d8235ad3f)
- Elizaveta Semenova의 (https://github.com/elizavetasemenova/PPLs/blob/master/ordered_logistic_regression/ordered_logistic_regression_PyMC3.ipynb)  

In [1]:
import pandas as pd 
import numpy as np
import pymc3 as pm
from sklearn.metrics import confusion_matrix, accuracy_score

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\ProgramData\Anaconda3\envs\gpu_test\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


WVS 데이터는 World Values Survey 에서 응답한 결과들을 바탕으로 만들어지 data이다.   
정확한 성격은 잘 모르지만, y 레이블이 1,2,3 으로 되어있다.

In [14]:
df = pd.read_csv('../dataset/WVS.csv')

In [15]:
df.y.unique()

array([1, 2, 3], dtype=int64)

In [16]:
df.head(3)

Unnamed: 0  religionno  religionyes  degreeyes  countryNorway  \
0           1           0            1          0              0   
1           2           0            1          0              0   
2           3           0            1          0              0   

   countrySweden  countryUSA  gendermale    age_sc  y  
0              0           1           1 -0.060858  1  
1              0           1           0 -0.294687  2  
2              0           1           0 -0.528515  1

In [17]:
## 불필요컬럼 drop
df.drop(columns='Unnamed: 0',inplace=True,axis=1)

In [20]:
print(df.shape)
df.head(2)

(5381, 9)


religionno  religionyes  degreeyes  countryNorway  countrySweden  \
0           0            1          0              0              0   
1           0            1          0              0              0   

   countryUSA  gendermale    age_sc  y  
0           1           1 -0.060858  1  
1           1           0 -0.294687  2

In [21]:
y = df.iloc[:,-1]
X = df.iloc[:, df.columns != 'y' ] ## 'y' 컬럼제외하고, 데이터셋구성
X = X.values ## chage to numpy type
y = y.values ## chage to numpy type

In [22]:
print(X.shape,y.shape)

(5381, 8) (5381,)


## Model specification

In [23]:
print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.9.3


In [24]:
N = X.shape[0] ## 5381
D = X.shape[1] ## 8

In [26]:
with pm.Model() as mod_test:
    
    # Priors
    sigma = pm.HalfNormal('sigma', sd = 1) 
    print(sigma)

sigma


In [ ]:
with pm.Model() as mod:
    
    # Priors
    sigma = pm.HalfNormal('sigma', sd = 1)   
    
    beta = pm.Normal('beta', mu=0, sd=sigma, shape=D) 
    
    lp = pm.math.dot(X, beta)
    
    cutpoints = pm.Normal("cutpoints", mu=[-0.01,0], sd=20, shape=2,transform=pm.distributions.transforms.ordered)
    
    # Likelihood 
    y_obs = pm.OrderedLogistic("y_obs", eta=lp, cutpoints=cutpoints, observed=y-1)